<a href="https://colab.research.google.com/github/gyasifred/msc-thesis/blob/main/BLEU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The notebook demonstrate sample implementation of [BLEU (Papineni et al., 2001)]([https://aclanthology.org/P02-1040/) using the python [NLTK](https://www.nltk.org/api/nltk.translate.bleu_score.html) package

# installl dependencies

In [ ]:
! pip install nltk
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
!pip install sacremoses
!pip3 install googletrans==3.1.0a0 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 6.4 MB/s 
     |████████████████████████████████| 72 kB 1.9 MB/s 
     |████████████████████████████████| 5.3 MB 61.3 MB/s 
     |████████████████████████████████| 163 kB 73.9 MB/s 
     |████████████████████████████████| 212 kB 56.7 MB/s 
     |████████████████████████████████| 115 kB 72.4 MB/s 
     |████████████████████████████████| 127 kB 72.0 MB/s 
     |████████████████████████████████| 115 kB 69.2 MB/s 
     |████████████████████████████████| 7.6 MB 41.7 MB/s 
     |████████████████████████████████| 1.3 MB 55.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg

## import dependencies

In [ ]:
import nltk
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
import time
import warnings
warnings.simplefilter('ignore')

## Google API BLEU

The implementation below will output BLEU score for translation using the google API. The example dataset is English-Twi parallel corpora. The first implementation involves getting BLEU score for 2-grams, 3-grams, and 4-grams. 
the other implemtation implemtation uses a weight matrix (0.58, 0, 0, 0).

Define GOOGLE API translator CLASS

In [ ]:
# Use google API for translation
# import libraries
from googletrans import Translator, constants
# instantiate a translator object
# initiate translator object
translator = Translator()
# Add Akan to the language supported by this package
# Note the googletrans package has not  been updated to capture the new additions by google since May 2022
# from https://translate.google.com/?sl=en&tl=ak&op=translate , the key and value for Twi is 'ak':'akan'
constants.LANGUAGES['ak'] = 'akan'

# PIVOT TRANSLATION
class GooglePivot:
    '''add dockstrings'''
    def __init__(self):
        pass

    def evaluate(self, sentences, src_key, dest_key):
        eng_text = translator.translate(sentences, src=src_key, dest='en').text
        print(eng_text)
        output = translator.translate(eng_text, dest=dest_key).text

        return output

# DIRECT TRANSLATION
class GoogleDirect:
    '''add dockstrings'''
    def __init__(self):
        pass

    def evaluate(self, sentences, src_key, dest_key):

        return translator.translate(sentences, src=src_key, dest=dest_key).text

Define BLEU CLASS

In [ ]:
class GoogleBleu():
    '''ADD DOCKSTRINGS'''
  
    def __init__(self, translator):
        self.translator = translator

    def get_bleuscore(self, testfile, referencefile, src_lang, dest_lang, smothingfunction=None):
        if type(testfile) == str and type(referencefile) == str:
            # Open test file and read lines
            f = open(testfile, "r")
            hypothesis = f.readlines()
            f.close()
            # open refernce file and read lines
            f = open(referencefile, "r")
            reference = f.readlines()
            f.close()
        elif type(testfile) == list and type(referencefile) == list:
            hypothesis = testfile
            reference = referencefile
        else:
            print(f'File must be txt or python list')

        # check the length of our input sentence
        length = len(hypothesis)
        bleu_total = np.array([0., 0., 0.])
        weights = [(1./2, 1./2), (1./3, 1./3, 1./3),
                   (1./4, 1./4, 1./4, 1./4)]
        for i in range(length):
            hypothesis[i] = hypothesis[i]
            reference[i] = reference[i]
            groundtruth = reference[i].lower().replace(" ' ", "'").replace(" .", ".").replace(" ?", "?").replace(" !", "!")\
                .replace(' " ', '" ').replace(' "', '"').replace(" : ", ": ").replace(" ( ", " (")\
                .replace(" ) ", ") ").replace(" , ", ", ").split()
            groundtruth = [groundtruth]
            translated_text = self.translator.evaluate(
                hypothesis[i], src_key=src_lang, dest_key=dest_lang)
            print("Translated Text: ", translated_text)
            print("Ground Truth: ", reference[i])
            candidate = translated_text.lower().replace(" ' ", "'").replace(" .", ".").replace(" ?", "?").replace(" !", "!")\
                .replace(' " ', '" ').replace(' "', '"').replace(" : ", ": ").replace(" ( ", " (")\
                .replace(" ) ", ") ").replace(" , ", ", ").split()
            bleu = np.array(sentence_bleu(
                groundtruth, candidate, weights, smoothing_function=smothingfunction))
            bleu_total += bleu

        return f'2-GRAMS: {bleu_total[0]/length}', f'3-GRAMS: {bleu_total[1]/length}', f'4-GRAMS: {bleu_total[2]/length}'

In [ ]:
class GoogleAPIBleu():
    def __init__(self, translator):
        self.translator = translator

    def get_bleuscore(self, testfile, referencefile, src_lang, dest_lang, smothingfunction=None):
        if type(testfile) == str and type(referencefile) == str:
            # Open test file and read lines
            f = open(testfile, "r")
            hypothesis = f.readlines()
            f.close()
            # open refernce file and read lines
            f = open(referencefile, "r")
            reference = f.readlines()
            f.close()
        elif type(testfile) == list and type(referencefile) == list:
            hypothesis = testfile
            reference = referencefile
        else:
            print(f'File must be txt or python list')

        # check the length of our input sentence
        length = len(hypothesis)
        bleu_total = 0
        weights = (0.58, 0, 0, 0)
        for i in range(length):
            hypothesis[i] = hypothesis[i]
            reference[i] = reference[i]
            groundtruth = reference[i].lower().replace(" ' ", "'").replace(" .", ".").replace(" ?", "?").replace(" !", "!")\
                .replace(' " ', '" ').replace(' "', '"').replace(" : ", ": ").replace(" ( ", " (")\
                .replace(" ) ", ") ").replace(" , ", ", ").split()
            groundtruth = [groundtruth]
            translated_text = self.translator.evaluate(
                hypothesis[i], src_key=src_lang, dest_key=dest_lang)
            print("Translated Text: ", translated_text)
            print("Ground Truth: ", reference[i])
            candidate = translated_text.lower().replace(" ' ", "'").replace(" .", ".").replace(" ?", "?").replace(" !", "!")\
                .replace(' " ', '" ').replace(' "', '"').replace(" : ", ": ").replace(" ( ", " (")\
                .replace(" ) ", ") ").replace(" , ", ", ").split()
            bleu = sentence_bleu(
                groundtruth, candidate, weights, smoothing_function=smothingfunction, auto_reweigh=True)
            bleu_total += bleu

        return f'BLEU SCORE: {bleu_total/length}'

## try it out

In [ ]:
# instantiate google direct translator
google = GoogleDirect()
# instantiate a smoothing function
smooth = SmoothingFunction()
# instantiate googlebleu class
bleu = GoogleBleu(google)
# get BLEU SCORE
# INPUT french OUTPUT akan(twi)
bleu.get_bleuscore("/content/english_test.txt","/content/twi_test.txt",src_lang='en', 
                   dest_lang='ak', smothingfunction=smooth.method7) # you can choose diffenrt smoothing function of your choice

Translated Text:  Dɛn na ɛrekɔ so wɔ ha?
Ground Truth:  Ɛdeɛn na  ɛrekɔso wɔ aha?

Translated Text:  Nyane
Ground Truth:  Sɔre

Translated Text:  Ɔba ha Fida biara
Ground Truth:  Ɔba ha Fiada biara

Translated Text:  Sua sɛ wobɛyɛ onyansafo
Ground Truth:  Sua nyansa

Translated Text:  Na minsusuw sɛ wobɛhwere wo kwan
Ground Truth:  Mannwene da sɛ wo bɛyera

Translated Text:  Sɛ wopɛ sɛ wode kasamu ahorow bɛboro anum hyɛ mu ma english a
Ground Truth:  Sɛ wo pɛ sɛ wo tintim brɔfo ne twi nsɛmfua nnum ne akyire a

Translated Text:  Me mma no rehaw me.
Ground Truth:  Me mma reha m'adwene

Translated Text:  Mene Simeon
Ground Truth:  Me ne Simeon

Translated Text:  Susuw wo ho hwɛ
Ground Truth:  Dwene wo ho

Translated Text:  Eyi yɛ asɛmpa
Ground Truth:  Eyi yɛ asɛm papa

Translated Text:  Fa saa asɛm no ma wo nua
Ground Truth:  Fa ɛno ma wo nua barima no

Translated Text:  medɔ wo
Ground Truth:  Me dɔ wo

Translated Text:  Ɔdɔ nkutoo ne nea yehia
Ground Truth:  Ɔdɔ nko ara na yɛ hia

Transl

('2-GRAMS: 0.4039621393222438',
 '3-GRAMS: 0.2855312663968064',
 '4-GRAMS: 0.1992943488796519')

try it out on GoogleAPIBleu, BLEU implementation as used by Azunre et al, (2021) in the paper [English-Twi Parallel Corpus for Machine Translation](https://arxiv.org/pdf/2103.15625.pdf)

In [ ]:
# instantiate google direct translator
google = GoogleDirect() 
# instantiate a smoothing function
smooth = SmoothingFunction()
# instantiate googlebleu class
bleu = GoogleAPIBleu(google)
# get BLEU SCORE
# INPUT french OUTPUT akan(twi)
bleu.get_bleuscore("/content/english_test.txt","/content/twi_test.txt",src_lang='en', 
                   dest_lang='ak', smothingfunction=smooth.method7) # you can choose diffenrt smoothing function of your choice

Translated Text:  Dɛn na ɛrekɔ so wɔ ha?
Ground Truth:  Ɛdeɛn na  ɛrekɔso wɔ aha?

Translated Text:  Nyane
Ground Truth:  Sɔre

Translated Text:  Ɔba ha Fida biara
Ground Truth:  Ɔba ha Fiada biara

Translated Text:  Sua sɛ wobɛyɛ onyansafo
Ground Truth:  Sua nyansa

Translated Text:  Na minsusuw sɛ wobɛhwere wo kwan
Ground Truth:  Mannwene da sɛ wo bɛyera

Translated Text:  Sɛ wopɛ sɛ wode kasamu ahorow bɛboro anum hyɛ mu ma english a
Ground Truth:  Sɛ wo pɛ sɛ wo tintim brɔfo ne twi nsɛmfua nnum ne akyire a

Translated Text:  Me mma no rehaw me.
Ground Truth:  Me mma reha m'adwene

Translated Text:  Mene Simeon
Ground Truth:  Me ne Simeon

Translated Text:  Susuw wo ho hwɛ
Ground Truth:  Dwene wo ho

Translated Text:  Eyi yɛ asɛmpa
Ground Truth:  Eyi yɛ asɛm papa

Translated Text:  Fa saa asɛm no ma wo nua
Ground Truth:  Fa ɛno ma wo nua barima no

Translated Text:  medɔ wo
Ground Truth:  Me dɔ wo

Translated Text:  Ɔdɔ nkutoo ne nea yehia
Ground Truth:  Ɔdɔ nko ara na yɛ hia

Transl

'BLEU SCORE: 0.635267969739873'

## BLEU SCORE USING HUGGING FACE OPUS-MT MODELS

Implement Bleu score base on the work of (Azunre et al., 2021) with a weigth matrix (0.58, 0, 0, 0), level 7 smoothing function and auto_reweigh=True

In [ ]:
class Bleu():
    def __init__(self, translator, tokenizer):
        self.translator = translator
        self.tokenizer = tokenizer

    def get_bleuscore(self, testfile, referencefile, smothingfunction=None):
        if type(testfile) == str and type(referencefile) == str:
            # Open test file and read lines
            f = open(testfile, "r")
            hypothesis = f.readlines()
            f.close()
            # open refernce file and read lines
            f = open(referencefile, "r")
            reference = f.readlines()
            f.close()
        elif type(testfile) == list and type(referencefile) == list:
            hypothesis = testfile
            reference = referencefile
        else:
            print(f'File must be txt or python list')

        # check the length of our input sentence
        length = len(hypothesis)
        bleu_total = 0
        weights = (0.58, 0, 0, 0)
        for i in range(length):
            hypothesis[i] = hypothesis[i]
            reference[i] = reference[i]
            groundtruth = reference[i].lower().replace(" ' ", "'").replace(" .", ".").replace(" ?", "?").replace(" !", "!")\
                .replace(' " ', '" ').replace(' "', '"').replace(" : ", ": ").replace(" ( ", " (")\
                .replace(" ) ", ") ").replace(" , ", ", ").split()
            groundtruth = [groundtruth]

            translated_text = self.translator.generate(
                **self.tokenizer(hypothesis[i], return_tensors="pt", padding=True))
            translated = [self.tokenizer.decode(
                t, skip_special_tokens=True) for t in translated_text]
            candidate = str(translated)[1:-1][1:-1].replace(" ' ", "'").replace(" .", ".").replace(" ?", "?").replace(" !", "!")\
                .replace(' " ', '" ').replace(' "', '"').replace(" : ", ": ").replace(" ( ", " (")\
                .replace(" ) ", ") ").replace(" , ", ", ")
            candidate = candidate.lower().split()
            print("Translated Text: ",candidate)
            print("Ground Truth: ",groundtruth)
            bleu = sentence_bleu(
                groundtruth, candidate, weights, smoothing_function=smothingfunction,
                auto_reweigh=True)
            bleu_total += bleu

        return f'BLEU SCORE: {bleu_total/length}'


In [ ]:
from transformers import MarianMTModel, MarianTokenizer
# use pre=trained English to Twi model
tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-tw")
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-tw")
smooth = SmoothingFunction()
bleu = Bleu(model, tokenizer)
bleu.get_bleuscore("/content/english_test.txt","/content/twi_test.txt", smooth.method7)

Translated Text:  ['dɛn', 'na', 'ɛrekɔ', 'so', 'wɔ', 'ha?']
Ground Truth:  [['ɛdeɛn', 'na', 'ɛrekɔso', 'wɔ', 'aha?']]
Translated Text:  ["w'abam", 'bebu']
Ground Truth:  [['sɔre']]
Translated Text:  ['ɔbaa', 'ha', 'fida', 'biara']
Ground Truth:  [['ɔba', 'ha', 'fiada', 'biara']]
Translated Text:  ['sua', 'sɛ', 'wobɛyɛ', "w'ade", 'nyansam']
Ground Truth:  [['sua', 'nyansa']]
Translated Text:  ['ná', 'misusuw', 'sɛ', 'wode', 'wo', 'nan', 'besi', 'fam']
Ground Truth:  [['mannwene', 'da', 'sɛ', 'wo', 'bɛyera']]
Translated Text:  ['sɛ', 'wopɛ', 'sɛ', 'wode', 'wo', 'ho', 'hyɛ', 'afiase', 'bɛboro', 'mfe', 'anum', 'mu', 'a']
Ground Truth:  [['sɛ', 'wo', 'pɛ', 'sɛ', 'wo', 'tintim', 'brɔfo', 'ne', 'twi', 'nsɛmfua', 'nnum', 'ne', 'akyire', 'a']]
Translated Text:  ['me', 'mma', 'no', 'rehaw', 'me.']
Ground Truth:  [['me', 'mma', 'reha', "m'adwene"]]
Translated Text:  ['mene', 'simeon']
Ground Truth:  [['me', 'ne', 'simeon']]
Translated Text:  ['susuw', 'wo', 'ho', 'hwɛ']
Ground Truth:  [['dwene', 

'BLEU SCORE: 0.5473762337573491'

# General Implementation


The general implementation takes in two argument, the translated test and the reference unlike the two above

In [ ]:
class BleuScore():
    def __init__(self):
      pass
    def get_bleuscore(self, testfile, referencefile, smothingfunction=None):
        if type(testfile) == str and type(referencefile) == str:
            # Open test file and read lines
            f = open(testfile, "r")
            hypothesis = f.readlines()
            f.close()
            # open refernce file and read lines
            f = open(referencefile, "r")
            reference = f.readlines()
            f.close()
        elif type(testfile) == list and type(referencefile) == list:
            hypothesis = testfile
            reference = referencefile
        else:
            print(f'File must be text file or python list')

        # check the length of our input sentence
        length = len(hypothesis)
        bleu_total = 0
        weights = (0.58, 0, 0, 0)
        for i in range(length):
            hypothesis[i] = hypothesis[i]
            reference[i] = reference[i]
            groundtruth = reference[i].lower().replace(" ' ", "'").replace(" .", ".").replace(" ?", "?").replace(" !", "!")\
                .replace(' " ', '" ').replace(' "', '"').replace(" : ", ": ").replace(" ( ", " (")\
                .replace(" ) ", ") ").replace(" , ", ", ").split()
            groundtruth = [groundtruth]

            candidate = hypothesis[i].lower().replace(" ' ", "'").replace(" .", ".").replace(" ?", "?").replace(" !", "!")\
                .replace(' " ', '" ').replace(' "', '"').replace(" : ", ": ").replace(" ( ", " (")\
                .replace(" ) ", ") ").replace(" , ", ", ").split()
            print("Translated Text: ",candidate)
            print("Ground Truth: ",groundtruth)
            bleu = sentence_bleu(
                groundtruth, candidate, weights, smoothing_function=smothingfunction,
                auto_reweigh=True)
            bleu_total += bleu

        return f'BLEU SCORE: {bleu_total/length}'


## try it out

First get some translations with google API from Akan(Twi) to English

In [ ]:
!git clone https://github.com/gyasifred/kasafranse.git

Cloning into 'kasafranse'...
remote: Enumerating objects: 172, done.
remote: Counting objects: 100% (172/172), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 172 (delta 107), reused 111 (delta 50), pack-reused 0
Receiving objects: 100% (172/172), 61.14 KiB | 10.19 MiB/s, done.
Resolving deltas: 100% (107/107), done.


In [ ]:
%cd kasafranse/

/content/kasafranse


In [ ]:
! ls

build_sub_word_tokenizer.py	      seq2seq_bleu.py
fine_turned_bleu.py		      seq2seq_nmt.ipynb
fine_turned_sacredbleu.py	      seq2seq_translate.py
fine_turned_transformer.py	      text_vectorization.py
fine_turned_transformer_translate.py  train_seq2seq_nmt.py
googleAPIdirect_translate.py	      train_test_split.py
googleAPIpivot_translate.py	      train_transformer.py
googleapisacrebleu.py		      transformer_bleu.py
__init__.py			      transformer.py
kasafranse			      transformer_sacred_bleu.py
README.md			      transformer_translate.py
requirements.txt


In [ ]:
!python googleAPIpivot_translate.py -h

usage: googleAPIpivot_translate.py [-h] [--output_name OUTPUT_NAME]
                                   [--output_dir OUTPUT_DIR]
                                   [--to_console TO_CONSOLE]
                                   file src_key dest_key

Pivot Translations with Google Translate API, pivot language :English

positional arguments:
  file                  Provide the file to be translated. This file must be a
                        txt file
  src_key               Provide the google translate API key for the source
                        language
  dest_key              Provide the google translate API key for the
                        destination language

optional arguments:
  -h, --help            show this help message and exit
  --output_name OUTPUT_NAME
                        Pass the name of the final output
  --output_dir OUTPUT_DIR
                        Pass the directory of the final output
  --to_console TO_CONSOLE
                        Specify whether to pri

In [ ]:
!python googleAPIpivot_translate.py \
/content/twi_test.txt \
ak \
en \
--output_name translate.txt 

In [ ]:
bleu = BleuScore()
bleu.get_bleuscore("/content/kasafranse/translate.txt","/content/english_test.txt", smooth.method7)

Translated Text:  ['what’s', 'going', 'on', 'in', 'aha?']
Ground Truth:  [['what', 'is', 'going', 'on', 'here?']]
Translated Text:  ['rise']
Ground Truth:  [['wake', 'up']]
Translated Text:  ['he', 'comes', 'here', 'every', 'friday']
Ground Truth:  [['she', 'comes', 'here', 'every', 'friday']]
Translated Text:  ['learn', 'wisdom']
Ground Truth:  [['learn', 'to', 'be', 'wise']]
Translated Text:  ['i', 'never', 'thought', 'you’d', 'get', 'lost']
Ground Truth:  [['i', 'didn’t', 'think', 'you', 'would', 'loose', 'your', 'way']]
Translated Text:  ['if', 'you', 'want', 'to', 'print', 'five', 'or', 'more', 'blue', 'and', 'white', 'words']
Ground Truth:  [['if', 'you', 'like', 'to', 'enter', 'more', 'than', 'five', 'pair', 'of', 'sentences', 'for', 'english']]
Translated Text:  ['my', 'kids', 'are', 'hurting', 'my', 'mind']
Ground Truth:  [['my', 'kids', 'are', 'worrying', 'me.']]
Translated Text:  ['i', 'am', 'simeon']
Ground Truth:  [['i', 'am', 'simeon']]
Translated Text:  ['think', 'about'

'BLEU SCORE: 0.719585085332997'

Additionally, you can implements General BLEU score for 2-grams, 3-grams, and 4-grams

In [ ]:
class BLEU():
    '''ADD DOCKSTRINGS'''
  
    def __init__(self):
      pass

    def get_bleuscore(self, testfile, referencefile, smothingfunction=None):
        if type(testfile) == str and type(referencefile) == str:
            # Open test file and read lines
            f = open(testfile, "r")
            hypothesis = f.readlines()
            f.close()
            # open refernce file and read lines
            f = open(referencefile, "r")
            reference = f.readlines()
            f.close()
        elif type(testfile) == list and type(referencefile) == list:
            hypothesis = testfile
            reference = referencefile
        else:
            print(f'File must be txt or python list')

        # check the length of our input sentence
        length = len(hypothesis)
        bleu_total = np.array([0., 0., 0.])
        weights = [(1./2, 1./2), (1./3, 1./3, 1./3),
                   (1./4, 1./4, 1./4, 1./4)]
        for i in range(length):
            hypothesis[i] = hypothesis[i]
            reference[i] = reference[i]
            groundtruth = reference[i].lower().replace(" ' ", "'").replace(" .", ".").replace(" ?", "?").replace(" !", "!")\
                .replace(' " ', '" ').replace(' "', '"').replace(" : ", ": ").replace(" ( ", " (")\
                .replace(" ) ", ") ").replace(" , ", ", ").split()
            groundtruth = [groundtruth]

            candidate = hypothesis[i].lower().replace(" ' ", "'").replace(" .", ".").replace(" ?", "?").replace(" !", "!")\
                .replace(' " ', '" ').replace(' "', '"').replace(" : ", ": ").replace(" ( ", " (")\
                .replace(" ) ", ") ").replace(" , ", ", ").split()
            print("Translated Text: ",candidate)
            print("Ground Truth: ",groundtruth)
            bleu = np.array(sentence_bleu(
                groundtruth, candidate, weights,auto_reweigh=True, smoothing_function=smothingfunction))
            bleu_total += bleu

        return f'2-GRAMS: {bleu_total[0]/length}', f'3-GRAMS: {bleu_total[1]/length}', f'4-GRAMS: {bleu_total[2]/length}'

In [ ]:
bleu = BLEU()
bleu.get_bleuscore("/content/kasafranse/translate.txt","/content/english_test.txt", smooth.method7)

Translated Text:  ['what’s', 'going', 'on', 'in', 'aha?']
Ground Truth:  [['what', 'is', 'going', 'on', 'here?']]
Translated Text:  ['rise']
Ground Truth:  [['wake', 'up']]
Translated Text:  ['he', 'comes', 'here', 'every', 'friday']
Ground Truth:  [['she', 'comes', 'here', 'every', 'friday']]
Translated Text:  ['learn', 'wisdom']
Ground Truth:  [['learn', 'to', 'be', 'wise']]
Translated Text:  ['i', 'never', 'thought', 'you’d', 'get', 'lost']
Ground Truth:  [['i', 'didn’t', 'think', 'you', 'would', 'loose', 'your', 'way']]
Translated Text:  ['if', 'you', 'want', 'to', 'print', 'five', 'or', 'more', 'blue', 'and', 'white', 'words']
Ground Truth:  [['if', 'you', 'like', 'to', 'enter', 'more', 'than', 'five', 'pair', 'of', 'sentences', 'for', 'english']]
Translated Text:  ['my', 'kids', 'are', 'hurting', 'my', 'mind']
Ground Truth:  [['my', 'kids', 'are', 'worrying', 'me.']]
Translated Text:  ['i', 'am', 'simeon']
Ground Truth:  [['i', 'am', 'simeon']]
Translated Text:  ['think', 'about'

('2-GRAMS: 0.5267301489893063',
 '3-GRAMS: 0.40760664316821',
 '4-GRAMS: 0.3581486114779136')